# Install Google Earth Engine and Follium

In [40]:
!pip install earthengine-api --upgrade
!pip install folium

Requirement already up-to-date: earthengine-api in c:\users\schne\anaconda3\lib\site-packages (0.1.384)


# Access Google Earth Engine Explorer

In [41]:
# import geopandas as gpd
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

# Load the GeoJSON File

In [42]:
gdf = gpd.read_file('buffered_stations (1).geojson')

# Install Google Cloud Storage

In [5]:
pip install google-cloud-storage

Note: you may need to restart the kernel to use updated packages.


# Cycle Through Images and Save to Google Cloud Storage

In [15]:
import pyautogui
import time
from datetime import datetime
import os
from ipywidgets import interact, widgets
from IPython.display import display as ipy_display, clear_output
import folium
from folium import Marker, Circle, CircleMarker
import pandas as pd
import time
from google.cloud import storage
from io import BytesIO

def wait_until_loaded(x, y, expected_color, timeout=10):
    """
    Wait until a pixel at (x, y) becomes the expected color.
    """
    start_time = time.time()
    while True:
        if pyautogui.pixelMatchesColor(x, y, expected_color):
            return True
        if time.time() - start_time > timeout:
            return False
        time.sleep(0.5)

# Setup GCS authentication
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\Schne\missing-lon-lat-project-dbd64a6ceebb.json"

# Initialize the GCS client
client = storage.Client()

# Define your bucket name
bucket_name = "tod"
bucket = client.bucket(bucket_name)

# Function to save the screenshot directly to GCS
def save_screenshot(facility_id):
    print(f"Inside save_screenshot with Facility ID: {facility_id}")  # Debug print

    # Use the Facility ID as the filename
    filename = f"{facility_id}.jpg"

    # Capture the screenshot to a byte stream
    screenshot = pyautogui.screenshot(region=(1300, 641, 1700, 1500))
    byte_stream = BytesIO()
    screenshot.save(byte_stream, format="JPEG")

    # Reset the stream position to the beginning
    byte_stream.seek(0)
    
    # Upload the byte stream to GCS
    blob = bucket.blob(filename)
    blob.upload_from_file(byte_stream, content_type="image/jpeg")
    print(f"Screenshot uploaded to GCS as {filename}")

def plot_map(desired_facility_id):
    if not desired_facility_id:  # Check if the input is empty
        print("Please enter a valid Facility ID.")
        return

    try:
        # Convert the input to integer and filter the GeoDataFrame
        facility_id = int(desired_facility_id)
        filtered_gdf = gdf[gdf['Facility ID'] == facility_id]

        # Proceed only if the Facility ID exists in the data
        if filtered_gdf.empty:
            print("Facility ID not found.")
            return

        # Extract the lat and lon from the filtered GeoDataFrame
        lat = filtered_gdf.iloc[0]['geometry'].centroid.y
        lon = filtered_gdf.iloc[0]['geometry'].centroid.x
    except ValueError:
        print("Invalid input. Please enter a numeric Facility ID.")
        return

    # Set up the map centered around the station
    m = folium.Map(location=[lat, lon], zoom_start=16, control_scale=True)

    # Add a small red dot at the center of the circle
    CircleMarker(location=[lat, lon], radius=3, color='red', fill=True, fill_color='red').add_to(m)

    # Add a circle of radius 1/4 mile (in meters) around the station
    Circle(location=[lat, lon], radius=402.336).add_to(m)

    # Add a layer control panel to the map
    m.add_child(folium.LayerControl())

    # Explicitly display the map
    ipy_display(m)

# Read the .csv file into a DataFrame
df = pd.read_csv('MTA_NYCT1.csv')

# Loop through the DataFrame to get facility IDs
for index, row in df.iterrows():
    clear_output(wait=True) 
    facility_id = row['Facility ID']  # Assuming the column name is 'Facility ID'
    print(f"Processing Facility ID: {facility_id}")  # Debug print
    plot_map(facility_id)

    # Wait until the image is loaded, e.g., wait until pixel (1400, 700) becomes white
    if wait_until_loaded(1400, 700, (255, 255, 255)):
        save_screenshot(facility_id)
    else:
        print(f"Timed out waiting for Facility ID: {facility_id}")
    
    # Save the screenshot using the Facility ID as the name
    save_screenshot(facility_id)

    # Optional: wait for some time before the next iteration (e.g., 5 seconds)
    time.sleep(5)


Processing Facility ID: 1985


Inside save_screenshot with Facility ID: 1985
Screenshot uploaded to GCS as 1985.jpg
Inside save_screenshot with Facility ID: 1985
Screenshot uploaded to GCS as 1985.jpg


KeyboardInterrupt: 

# Code to bring up one image at a time based on Facility ID

In [50]:
import pyautogui
import time
import os
from ipywidgets import interact, widgets
from IPython.display import display as ipy_display, clear_output
import folium
from folium import CircleMarker, Circle
import pandas as pd

def plot_map(desired_facility_id, gdf):
    try:
        # Check if 'Facility ID' column exists
        if 'Facility ID' not in gdf.columns:
            print("Facility ID column not found in the data.")
            return

        # Convert the input to integer
        facility_id = int(desired_facility_id)
        filtered_gdf = gdf[gdf['Facility ID'] == facility_id]

        # Proceed only if the Facility ID exists in the data
        if filtered_gdf.empty:
            print("Facility ID not found.")
            return

        # Extract the coordinates
        coords = filtered_gdf.iloc[0]['LonLat'].strip('()').split(', ')
        lat = float(coords[1])
        lon = float(coords[0])

    except ValueError:
        print("Invalid input. Please enter a numeric Facility ID.")
        return

    # Set up the map centered around the station
    m = folium.Map(location=[lat, lon], zoom_start=16, control_scale=True)

    # Add a small red dot at the center of the circle
    CircleMarker(location=[lat, lon], radius=3, color='red', fill=True, fill_color='red').add_to(m)

    # Add a circle of radius 1/4 mile (in meters) around the station
    Circle(location=[lat, lon], radius=402.336).add_to(m)

    # Add a layer control panel to the map
    m.add_child(folium.LayerControl())

    # Explicitly display the map
    ipy_display(m)

# Read the .csv file into a DataFrame
gdf = pd.read_csv('MTA_NYCT1.csv')

# Create an interactive widget for Facility ID input
facility_id_input = widgets.Text(
    value='',
    placeholder='Enter Facility ID',
    description='Facility ID:',
    disabled=False
)

# Create a button to trigger the map plotting
plot_button = widgets.Button(description="Show Map")

# Define the button click event handler
def on_button_clicked(b):
    clear_output(wait=True)
    plot_map(facility_id_input.value, gdf)

# Attach the event handler to the button
plot_button.on_click(on_button_clicked)

# Display the widgets
ipy_display(facility_id_input, plot_button)


# END